In [24]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/9 juni/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 45
class_id = 106

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 45 class 106.xlsx")


file_to_subject = {}
teacher_tag = '0x24046130BA41'

for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:

        subject_id = keyfile_specific.loc[keyfile_specific['tagnr.'] == tagnr, 'subject ID'].values[0]
    except:
        print("Tag not found:" + tag_id)
        print(tagnr)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tag not found:0x24025F449EFB
30


In [25]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-25-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-25-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-25-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df

In [26]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
               TimeStamp         X        Y         Z
2957 2023-09-06 09:53:46  1.736977  4.32333  0.525989
2958 2023-09-06 09:53:47  1.781819  4.32333  0.325595
2959 2023-09-06 09:53:48  1.347282  4.32333  0.515204
2960 2023-09-06 09:53:49  0.645829  4.32333  0.448031
2961 2023-09-06 09:53:50 -0.321879  4.32333  0.653395
2962 2023-09-06 09:53:51 -0.443089  4.32333  0.556116
2963 2023-09-06 09:53:52 -0.443089  4.32333  0.496549
2964 2023-09-06 09:53:53 -0.443089  4.32333  0.702621
2965 2023-09-06 09:53:54 -0.120354  4.32333  0.735125
2966 2023-09-06 09:53:55  0.500656  4.32333  0.495285
2967 2023-09-06 09:53:56  1.094255  4.32333  0.561753

Maximum value of X:
               TimeStamp         X        Y         Z
4956 2023-09-06 10:27:05  3.722385  4.32333  0.518293
4957 2023-09-06 10:27:06  4.766803  4.32333  0.779252
4958 2023-09-06 10:27:07  5.269489  4.32333  0.516590
4959 2023-09-06 10:27:08  6.562253  4.32333  0.499676
4960 2023-09-06 10:27:09  6.562253  4.323

In [28]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}
teacher_tag = '0x24046130BA41'
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr.'] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_21.0 with Tagnr 7      4.692834  4.314643  0.443104
Person_7.0 with Tagnr 13      0.573253  1.182295  0.546108
Person_6.0 with Tagnr 9       3.119522  3.263944  0.435559
Person_18.0 with Tagnr 15     0.862077  1.790650  0.515813
Person_20.0 with Tagnr 6      0.540588  5.744200  0.404987
Person_Teacher with Tagnr 35  2.273004  6.844682  0.512171
Person_11.0 with Tagnr 3      0.878608  1.626838  0.509511
Person_16.0 with Tagnr 17     3.962425  5.709020  0.539839
Person_13.0 with Tagnr 2      4.008730  4.289075  0.477323
Person_14.0 with Tagnr 20     3.274024  4.590659  0.521997
Person_15.0 with Tagnr 10     1.860404  4.515238  0.425774
Person_4.0 with Tagnr 11      1.051293  4.219448  0.396284
Person_8.0 with Tagnr 18      4.011899  2.974414  0.427823
Person_12.0 with Tagnr 19     6.214368  1.304967  0.575409
Person_19.0 with Tagnr 5      5.712615  4.188169  0.494226


In [29]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
7   3.688441  4.411123  0.484019
13  0.498266  1.396069  0.705373
9   4.144845  3.703126  0.348177
15  1.104062  2.051709  0.401905
6   6.400648  5.569304  0.430551
35  1.385345  6.591261  0.483050
3   5.709635  3.974325  0.616766
17  4.596302  4.347315  0.582135
2   6.389053  5.239175  0.413497
20  5.811392  2.225382  0.544916
10  1.654879  4.237932  0.323351
11  1.162725  4.278494  0.294221
18  4.083811  3.116904  0.454116
19  6.025626  1.293246  0.568428
5   5.597801  2.762447  0.471423


In [30]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
7   3.472671  4.236352  0.887055
13  0.283240  2.596339  0.628819
9   6.312008  4.810241  0.586574
15  1.038495  2.117581  0.226208
6   2.979148  3.241866  0.619799
35  1.713468  5.051968  0.478088
3   5.710215  3.912743  0.641169
17  4.647711  1.839120  0.478894
2   6.366040  4.079705  0.239544
20  5.771803  2.290152  0.370985
10  1.683389  4.073882  0.282826
11  1.005712  4.060604  0.270168
18  4.019988  2.975093  0.670515
19  5.981621  1.366643  0.344128
5   2.370666  3.503295  0.310504


In [31]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 35

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

3.1713176668423233 7
5.2703821708015575 13
3.9945167685711516 9
4.5482579763829705 15
5.118364994744695 6
5.054486958324147 3
3.9173388221191687 17
5.1831687042170635 2
6.216976370676369 20
2.3687142696836228 10
2.3234569080104626 11
4.399190771098077 18
7.0428102202890095 19
5.692504580899667 5
The closest tracker to the teacher is 11 with a distance of 2.3234569080104626


In [32]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher

for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within full distance of anyone: 29058
Person 16.0 with Tracker 17: 15280 times within full distance of teacher, category: above average with proximity percentage 52.58
Person 14.0 with Tracker 20: 13187 times within full distance of teacher, category: above average with proximity percentage 45.38
Person 13.0 with Tracker 2: 226 times within full distance of teacher, category: below average with proximity percentage 0.78
Person 15.0 with Tracker 10: 185 times within full distance of teacher, category: below average with proximity percentage 0.64
Person 20.0 with Tracker 6: 65 times within full distance of teacher, category: below average with proximity percentage 0.22
Person 4.0 with Tracker 11: 49 times within full distance of teacher, category: below average with proximity percentage 0.17
Person 21.0 with Tracker 7: 46 times within full distance of teacher, category: below average with proximity percentage 0.16
Person 8.0 with Tracker 18: 12 times within full d

In [33]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 232
Person 13.0 with Tracker 2: 166 times within half distance of teacher, category: above average with proximity percentage 71.55
Person 15.0 with Tracker 10: 35 times within half distance of teacher, category: above average with proximity percentage 15.09
Person 4.0 with Tracker 11: 23 times within half distance of teacher, category: above average with proximity percentage 9.91
Person 20.0 with Tracker 6: 6 times within half distance of teacher, category: below average with proximity percentage 2.59
Person 16.0 with Tracker 17: 1 times within half distance of teacher, category: below average with proximity percentage 0.43
Person 14.0 with Tracker 20: 1 times within half distance of teacher, category: below average with proximity percentage 0.43
Person 21.0 with Tracker 7: 0 times within half distance of teacher, category: below average with proximity percentage 0.00
Person 7.0 with Tracker 13: 0 times within half distance of tea

In [34]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 7
Person 15.0 with Tracker 10: 6 times within quarter distance of teacher, category: above average with proximity percentage 85.71
Person 4.0 with Tracker 11: 1 times within quarter distance of teacher, category: above average with proximity percentage 14.29
Person 21.0 with Tracker 7: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 7.0 with Tracker 13: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 6.0 with Tracker 9: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 18.0 with Tracker 15: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 20.0 with Tracker 6: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 11.0 with Tracker 3: 0 times within quar

In [36]:

merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31 (1) (1).xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31 (1) (1).xlsx", index=False)